# Big PV model

This is a simplified PV model which is used to rapidly and continuously forecast the potential and production of photovoltaic energy at the neighbourhood and city scales.

Background:
----------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms.

Later parametrized equations, namely placement efficiency and geometric efficiency are generated from the training areas, which were later evaluated using the evaluation areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Geometric efficiency calculates the PV power output with shading losses instead of using the SEBE model. 

These parametrized equations are used in Big PV model. With the placement and geometric efficiency, PV power output for an urban area can be calculated in a simple and fast manner.

Significance:
----------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.

Limitations:
----------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.


In [ ]:
    """

    Input Parameters
    ----------
        l_PV         : Length of PV panel in portait mode [m]
        w_PV         : Width of PV panel in portait mode [m]
        pv_eff       : Efficiency of PV panel [%]
        alpha        : Temperature coefficient of PV panel [1/K]
        d            : Module row spacing [m]
        lamda_proj   : Projected plan area fraction of building segments
        l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                       which tells you how far the shadows get on the roof and is calculated 
                       by taking the square root of the average segment area.
        f_flat       : Fraction of flat building segments
        wire_losses  : Wiring losses [%]
        metadata     : Input meteorological data specifically formatted using 
                       UMEP -> Pre-processing -> Meteorological data -> Prepare existing data

    """

In [ ]:
import numpy as np; 

l_PV = 1
w_PV = 1.6  
pv_eff = 17.7 
alpha = -0.0039  
d = 1.5  
lamda_proj = 0.24  
l_s = 8.88  
f_flat = 0.14  
wire_losses = 2 
metadata = np.loadtxt("Berlin_Weather_2019.txt", skiprows=1, dtype=float).reshape((-1, 24))

In [ ]:
  """
    Constants
    -------
        ref_temp     : Reference PV cell temperature, default value = 25 °C
        k_t          : Reduction factor, default value = 0.05 °C/(W/m2)
            
            Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
            differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
            A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
            (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
            building integrated systems.
            
            References:
            -------
            
            Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
            Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
            and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
            52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002
            
            King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.
            
            
  """

ref_temp = 25
k_t = 0.05

In [ ]:
  """
    Parametrized Equations
    -------
        pl_eff       : Placement Efficiency
        
                Placement Efficiency (PE) is the ratio of geometrically installed PV panels
                to the theoretical PV panels.
                PE for Big PV model is 0.897 (taken from Master Thesis)
                
        geo_eff      : Geometric Efficiency
        
                Geometric Efficiency (GE) is the ratio of PV power output (including shading losses) 
                to the theoretical PV power output.
                GE for Big PV model is calculated using the equation (0.0119 * l_s) + 0.4046 (taken from Master Thesis)
  """
    
pl_eff = 0.897
geo_eff = (0.0119 * l_s) + 0.4046

In [ ]:
  """
    Theoretical PV panels
    -------
        PV panels are calculated using rooftop building segment area without considering the geometry, tilt, 
        and aspect of the building segment.
        
  """
theoretical_PVP = (f_flat * lamda_proj / ((w_PV + d) * l_PV)) + ((1 - f_flat) * lamda_proj / (l_PV * w_PV))

In [ ]:
  """
    Geometric PV panels
    -------
        Geometrical distribution of PV panels on rooftop building segments are calculated 
        using Theoretical PV panels and placement efficiency
        
  """
geometric_PVP = theoretical_PVP * pl_eff

In [ ]:
def big_pv_flat(panels):
    
    """
    Returns
    -------
        Theoretical PV power output for an urban area over an given time period.[W/m2]
        
    """
    pv_power = 0
    for i in range(metadata.shape[0]):
        if metadata[[i], 14] > 0:
            G_s = metadata[[i], 14]
            pv_power = pv_power = pv_power + (G_s * (pv_eff / 100) * \
                                              (1 + alpha * ((metadata[[i], 11] + k_t * G_s) - ref_temp))\
                                              * l_PV * w_PV * panels * (1 - (wire_losses / 100)))
    
    pv_power = pv_power
    return pv_power

theoretical_PVO = big_pv_flat(geometric_PVP)

In [ ]:
"""
    Returns
    -------
        Theoretical PV power output for an urban area over an given time period.[W/m2]
        
"""
modelled_PVO = theoretical_PVO * geo_eff